In [1]:
import pandas as pd
import os

os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

from tensorflow.keras import layers, Sequential
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

2025-04-22 10:06:38.338045: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 10:06:38.341142: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 10:06:38.349277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745309198.362569   30017 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745309198.366110   30017 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745309198.376141   30017 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
columns = ['unit', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor{i}' for i in range(1, 22)]
df1 = pd.read_csv(
    '../dataset/train_FD001.txt', sep='\\s+', header=None, names=columns)
df2 = pd.read_csv(
    '../dataset/train_FD002.txt', sep='\\s+', header=None, names=columns)
df3 = pd.read_csv(
    '../dataset/train_FD003.txt', sep='\\s+', header=None, names=columns)
df4 = pd.read_csv(
    '../dataset/train_FD004.txt', sep='\\s+', header=None, names=columns)

df2['unit'] += df1['unit'].max()
df3['unit'] += df2['unit'].max()
df4['unit'] += df3['unit'].max()

df = pd.concat([df1, df2, df3, df4])

t1 = pd.read_csv(
    '../dataset/test_FD001.txt', sep='\\s+', header=None, names=columns)
t2 = pd.read_csv(
    '../dataset/test_FD002.txt', sep='\\s+', header=None, names=columns)
t3 = pd.read_csv(
    '../dataset/test_FD003.txt', sep='\\s+', header=None, names=columns)
t4 = pd.read_csv(
    '../dataset/test_FD004.txt', sep='\\s+', header=None, names=columns)

t2['unit'] += df1['unit'].max()
t3['unit'] += df2['unit'].max()
t4['unit'] += df3['unit'].max()

test_data = pd.concat([t1, t2, t3, t4])

r1 = pd.read_csv(
    '../dataset/RUL_FD001.txt', sep='\\s+', header=None, names=['RUL'])
r2 = pd.read_csv(
    '../dataset/RUL_FD002.txt', sep='\\s+', header=None, names=['RUL'])
r3 = pd.read_csv(
    '../dataset/RUL_FD003.txt', sep='\\s+', header=None, names=['RUL'])
r4 = pd.read_csv(
    '../dataset/RUL_FD004.txt', sep='\\s+', header=None, names=['RUL'])

rul_data = pd.concat([r1, r2, r3, r4])

for d in [[df, 'df'], [test_data, 'test_data'], [rul_data, 'rul_data']]:
    print('{}:'.format(d[1]))
    print(d[0].describe())
    print(d[0].info())


df:
                unit          cycle       setting1       setting2  \
count  160359.000000  160359.000000  160359.000000  160359.000000   
mean      370.270050     123.331338      17.211973       0.410004   
std       203.411987      83.538146      16.527988       0.367938   
min         1.000000       1.000000      -0.008700      -0.000600   
25%       196.000000      57.000000       0.001300       0.000200   
50%       381.000000     114.000000      19.998100       0.620000   
75%       547.000000     173.000000      35.001500       0.840000   
max       709.000000     543.000000      42.008000       0.842000   

            setting3        sensor1        sensor2        sensor3  \
count  160359.000000  160359.000000  160359.000000  160359.000000   
mean       95.724344     485.840890     597.361022    1467.035653   
std        12.359044      30.420388      42.478516     118.175261   
min        60.000000     445.000000     535.480000    1242.670000   
25%       100.000000     449.

In [3]:
df['RUL'] = df.groupby('unit')['cycle'].transform(lambda x: x.max() - x)

In [4]:
df.head()

,unit,cycle,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [5]:
units = df.groupby('unit')
datapoint_columns = df.drop(['unit', 'RUL'], axis=1).columns

df_smoothed = pd.DataFrame()
sequences = []
ruls = []

max_len = 80
for _, unit in units:
    unit_smoothed = unit.apply(lambda column: column.ewm(alpha=0.2, adjust=False).mean())
    df_smoothed = pd.concat([df_smoothed, unit_smoothed])
    
    for i in range(0, max_len):
        sequence = unit_smoothed[datapoint_columns].values[0:i+1]
        rul = unit_smoothed['RUL'].values[i]
        
        sequences.append(sequence)
        ruls.append(rul)

df = df_smoothed

print(sequences[0:3])
print(ruls[0:3])
print(max_len)

[array([[ 1.00000e+00, -7.00000e-04, -4.00000e-04,  1.00000e+02,
         5.18670e+02,  6.41820e+02,  1.58970e+03,  1.40060e+03,
         1.46200e+01,  2.16100e+01,  5.54360e+02,  2.38806e+03,
         9.04619e+03,  1.30000e+00,  4.74700e+01,  5.21660e+02,
         2.38802e+03,  8.13862e+03,  8.41950e+00,  3.00000e-02,
         3.92000e+02,  2.38800e+03,  1.00000e+02,  3.90600e+01,
         2.34190e+01]]), array([[ 1.000000e+00, -7.000000e-04, -4.000000e-04,  1.000000e+02,
         5.186700e+02,  6.418200e+02,  1.589700e+03,  1.400600e+03,
         1.462000e+01,  2.161000e+01,  5.543600e+02,  2.388060e+03,
         9.046190e+03,  1.300000e+00,  4.747000e+01,  5.216600e+02,
         2.388020e+03,  8.138620e+03,  8.419500e+00,  3.000000e-02,
         3.920000e+02,  2.388000e+03,  1.000000e+02,  3.906000e+01,
         2.341900e+01],
       [ 1.200000e+00, -1.800000e-04, -3.800000e-04,  1.000000e+02,
         5.186700e+02,  6.418860e+02,  1.590124e+03,  1.401108e+03,
         1.462000e+01,

In [6]:
sequences_padded = pad_sequences(sequences, maxlen=max_len, dtype='float32', padding='post')
ruls = np.array(ruls)

num_features = sequences_padded.shape[2]

model = Sequential([
    layers.Input(shape=(max_len, num_features)),
    
    # Masking layer so the LSTM ignores padded zeros
    layers.Masking(mask_value=0.),

    # LSTM layer (you can stack more if you want)
    layers.LSTM(64, return_sequences=False, recurrent_dropout=0.2),

    layers.Dropout(0.3),

    # Output layer: single value (RUL)
    layers.Dense(1)
])


def lr_scheduler(epoch, lr):
    if epoch <= 10:
        return 0.01  # High for first 5 epochs
    else:
        return 0.001  # Lower for the rest


model.compile(optimizer=Adam(learning_rate=0.004), loss='mse')

model.summary()

2025-04-22 10:07:51.696947: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 80, 25)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        23,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,105 (90.25 KB)

 Trainable params: 23,105 (90.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    sequences_padded,
    ruls,
    epochs=60,
    batch_size=80,
    validation_split=0.2,
)

2025-04-22 10:08:06.165544: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363008000 exceeds 10% of free system memory.


Epoch 1/60
 26/568 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - loss: 36290.6055

In [14]:
units = test_data.groupby('unit')

test_max_len = 0
test_sequences = []
for _, unit in units:
    sequence = unit[datapoint_columns].values
    if len(sequence) > test_max_len:
        test_max_len = len(sequence)
    test_sequences.append(sequence)

print(sequences[0:3])
print(max_len)

[array([[ 1.00000e+00, -7.00000e-04, -4.00000e-04,  1.00000e+02,
         5.18670e+02,  6.41820e+02,  1.58970e+03,  1.40060e+03,
         1.46200e+01,  2.16100e+01,  5.54360e+02,  2.38806e+03,
         9.04619e+03,  1.30000e+00,  4.74700e+01,  5.21660e+02,
         2.38802e+03,  8.13862e+03,  8.41950e+00,  3.00000e-02,
         3.92000e+02,  2.38800e+03,  1.00000e+02,  3.90600e+01,
         2.34190e+01]]), array([[ 1.0000000e+00, -7.0000000e-04, -4.0000000e-04,  1.0000000e+02,
         5.1867000e+02,  6.4182000e+02,  1.5897000e+03,  1.4006000e+03,
         1.4620000e+01,  2.1610000e+01,  5.5436000e+02,  2.3880600e+03,
         9.0461900e+03,  1.3000000e+00,  4.7470000e+01,  5.2166000e+02,
         2.3880200e+03,  8.1386200e+03,  8.4195000e+00,  3.0000000e-02,
         3.9200000e+02,  2.3880000e+03,  1.0000000e+02,  3.9060000e+01,
         2.3419000e+01],
       [ 1.0400000e+00, -5.9600000e-04, -3.9600000e-04,  1.0000000e+02,
         5.1867000e+02,  6.4183320e+02,  1.5897848e+03,  1.40

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

test_sequences_padded = pad_sequences(test_sequences, maxlen=test_max_len, dtype='float32', padding='post')

rul_pred = model.predict(test_sequences_padded)
ruls = rul_data['RUL']

val_rmse = np.sqrt(mean_squared_error(ruls, rul_pred))
val_mae = mean_absolute_error(ruls, rul_pred)
val_r2 = r2_score(ruls, rul_pred)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


In [17]:
print(val_rmse)
print(val_mae)
print(val_r2)

139.80775025459067
130.26817321777344
-6.493597984313965


In [27]:
test_sequences_padded.shape

(259, 1045, 25)